In [ ]:
'''
version 2
去除 face mask
'''

In [1]:
import tensorflow as tf
import os
import numpy as np
from mit_eye_v2 import mit_itraker
import time
sess=tf.InteractiveSession()

In [2]:
with tf.name_scope('PlaceHolder'):
    LeftEye=tf.placeholder(tf.float32,[None,64,64,3],name='LeftEye')
    RightEye=tf.placeholder(tf.float32,[None,64,64,3],'RightEye')
    FaceOri=tf.placeholder(tf.float32,[None,64,64,3],'FaceOri')
    #FaceMask=tf.placeholder(tf.float32,[None,25,25],'FaceMask')
    Y=tf.placeholder(tf.float32,[None,2],name='Y')
    LR=tf.placeholder(tf.float32,name='Learning_rate')
    REG_RATE=tf.placeholder(tf.float32,name='Reg_rate')
    DROPOUT=tf.placeholder(tf.float32,name='DropOut_rate')
    BN_FLAG=tf.placeholder(tf.bool,name='BatchNormalizeFlag')

In [3]:
with tf.name_scope('MIT_Model'):
    model=mit_itraker(r'D:\Data warehouse\Face_Picture\eye_tracker_train_and_val',left_eye=LeftEye,right_eye=RightEye,face_ori=FaceOri,\
                      dropout=DROPOUT,bn_train=BN_FLAG,reg_rate=REG_RATE)
    y_pre=model.coordinate

Building graph done!
Loading file~
Load Data Done~


In [4]:
with tf.name_scope('LOSS'):
    Loss=(tf.losses.mean_squared_error(Y,y_pre))+tf.add_n(tf.get_collection('loss_w'))
    tf.summary.scalar('loss',Loss)
    ERR=tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.squared_difference(y_pre, Y), axis=1)))
    tf.summary.scalar('err',ERR)
with tf.name_scope('Trainer'):
    Trainer=tf.train.AdamOptimizer(LR).minimize(Loss)

with tf.name_scope('Logger'):
    init=tf.global_variables_initializer()

    writer_tr=tf.summary.FileWriter('./mylog/train_v2',sess.graph)
    writer_te=tf.summary.FileWriter('./mylog/val_v2')
    merge=tf.summary.merge_all()

In [5]:
data=model.data
bn_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
'''
['val_face', 'train_eye_left', 'train_y', 'train_eye_right', 'val_y', 'train_face_mask', 'train_face',
'val_eye_right', 'val_face_mask', 'val_eye_left']
'''

"\n['val_face', 'train_eye_left', 'train_y', 'train_eye_right', 'val_y', 'train_face_mask', 'train_face',\n'val_eye_right', 'val_face_mask', 'val_eye_left']\n"

In [6]:
#重新训练
sess.run(init)
saver=tf.train.Saver()

In [ ]:
#加载保存的模型，继续训练
saver=tf.train.Saver()
#saver=tf.train.Saver()
saver.restore(sess,'./model_save/model_mit/model_err106.ckpt-44270')

In [ ]:
#130 个loop用时1分钟 无BN,dropout
#5000 个loop用时1h 有BN,dropout
batch_size=128
dp_=0.5
lr_=1e-3
rg_rate_=9e-5
s_time=time.time()
best_val_err=100
err_delta=0.1
for i_ in range(5000*2,5000*2+5000*10):
    mask=np.random.choice(range(48000),batch_size,replace=False)
    face_=data['train_face'][mask]
    left_eye_,right_eye_=data['train_eye_left'][mask],data['train_eye_right'][mask]
    y_=data['train_y'][mask]
    feed_dict={LeftEye:left_eye_,  RightEye:right_eye_,  FaceOri:face_,  \
              Y:y_,LR:lr_,  DROPOUT:dp_,  BN_FLAG:True,  REG_RATE:rg_rate_}
    sess.run([Trainer,bn_ops],feed_dict=feed_dict)
    if i_%5==0:
        feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,\
              Y:y_,LR:lr_,DROPOUT:1.,BN_FLAG:False,REG_RATE:rg_rate_}
        loss_,m_,_=sess.run([Loss,merge,bn_ops],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i_)
        #print('epoch:{},loss:{}'.format(i_,loss_))
    if i_%10==0:
        mask=np.random.choice(range(5000),batch_size,replace=False)
        face_=data['val_face'][mask]
        left_eye_,right_eye_=data['val_eye_left'][mask],data['val_eye_right'][mask]
        y_=data['val_y'][mask]
        feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,\
                  Y:y_,LR:lr_,DROPOUT:1.,BN_FLAG:False,REG_RATE:rg_rate_}
        loss_,err_,m_,_=sess.run([Loss,ERR,merge,bn_ops],feed_dict=feed_dict)
        writer_te.add_summary(m_,i_)
        print('--epoch:{},loss:{},error:{}'.format(i_,loss_,err_))
        #保存模型
        if err_<1.9:
            if err_+err_delta<best_val_err:
                best_val_err=err_
                err_val=str(int(err_*100))
                saver.save(sess,'./model_save/model_mit/model_err'+err_val+'.ckpt',global_step=i_)
                print('@@@epoch:{},best err:{},model save'.format(i_,best_val_err))
    if i_%(5000)==0:
        lr_=max(1e-5,lr_*0.9)
        print("**epoch:{},lr:{}".format(i_,lr_))
e_time=time.time()
run_time=e_time-s_time
print('Trainning Time:{:.0f} h,{:.0f} min,{:.2f} sec'.format(run_time//3600,(run_time%3600)//60,run_time%60))


--epoch:10000,loss:3.0666399002075195,error:1.7640161514282227
@@@epoch:10000,best err:1.7640161514282227,model save
**epoch:10000,lr:0.0009000000000000001
--epoch:10010,loss:3.271810531616211,error:1.873260259628296
--epoch:10020,loss:4.242954254150391,error:2.054915189743042
--epoch:10030,loss:3.826662302017212,error:1.9624476432800293
--epoch:10040,loss:4.800353050231934,error:2.089885711669922
--epoch:10050,loss:3.4767119884490967,error:1.8060280084609985
--epoch:10060,loss:3.5677847862243652,error:1.926957130432129
--epoch:10070,loss:4.581240653991699,error:2.0496132373809814
--epoch:10080,loss:3.038708209991455,error:1.7258720397949219
--epoch:10090,loss:3.218163013458252,error:1.796312928199768
--epoch:10100,loss:4.09910774230957,error:2.0245182514190674
--epoch:10110,loss:4.419067859649658,error:2.0666143894195557
--epoch:10120,loss:2.8261570930480957,error:1.7240180969238281
--epoch:10130,loss:3.7157766819000244,error:2.034106731414795
--epoch:10140,loss:2.7411625385284424,err

--epoch:11300,loss:3.3069863319396973,error:1.7908854484558105
--epoch:11310,loss:3.5067598819732666,error:1.785101294517517
--epoch:11320,loss:3.437870502471924,error:1.854590892791748
--epoch:11330,loss:3.9858877658843994,error:1.860612392425537
--epoch:11340,loss:3.48140025138855,error:1.8676587343215942
--epoch:11350,loss:3.126140594482422,error:1.718246579170227
--epoch:11360,loss:3.6760599613189697,error:1.928597331047058
--epoch:11370,loss:3.2069969177246094,error:1.908583641052246
--epoch:11380,loss:2.8431248664855957,error:1.5772345066070557
--epoch:11390,loss:4.281254291534424,error:2.047469139099121
--epoch:11400,loss:2.6940715312957764,error:1.6312114000320435
--epoch:11410,loss:3.8250269889831543,error:1.9826329946517944
--epoch:11420,loss:3.793055772781372,error:1.9458616971969604
--epoch:11430,loss:3.0891075134277344,error:1.7160720825195312
--epoch:11440,loss:2.7497196197509766,error:1.64523446559906
--epoch:11450,loss:3.9439172744750977,error:1.8308286666870117
--epoch

--epoch:12620,loss:3.9848735332489014,error:1.8894506692886353
--epoch:12630,loss:4.1678242683410645,error:1.9742423295974731
--epoch:12640,loss:3.864793300628662,error:1.955059289932251
--epoch:12650,loss:3.485276937484741,error:1.810652256011963
--epoch:12660,loss:3.09722900390625,error:1.6538141965866089
--epoch:12670,loss:3.7747702598571777,error:1.8776496648788452
--epoch:12680,loss:3.316953182220459,error:1.7246997356414795
--epoch:12690,loss:2.9260473251342773,error:1.6525450944900513
--epoch:12700,loss:3.1406378746032715,error:1.7166998386383057
--epoch:12710,loss:3.3627142906188965,error:1.7580503225326538
--epoch:12720,loss:3.8429770469665527,error:1.9441715478897095
--epoch:12730,loss:3.570040225982666,error:1.9363590478897095
--epoch:12740,loss:3.444446563720703,error:1.7290327548980713
--epoch:12750,loss:3.7293171882629395,error:1.9342246055603027
--epoch:12760,loss:3.5800271034240723,error:1.8745890855789185
--epoch:12770,loss:3.969106435775757,error:1.9506711959838867
--

--epoch:13940,loss:4.324697017669678,error:1.9316470623016357
--epoch:13950,loss:4.024379253387451,error:1.967291235923767
--epoch:13960,loss:4.277533054351807,error:1.946850299835205
--epoch:13970,loss:3.503812074661255,error:1.9089281558990479
--epoch:13980,loss:3.862635374069214,error:1.8955892324447632
--epoch:13990,loss:3.7186079025268555,error:1.903801441192627
--epoch:14000,loss:3.3568034172058105,error:1.806999921798706
--epoch:14010,loss:3.5286951065063477,error:1.8853427171707153
--epoch:14020,loss:3.183966875076294,error:1.7972782850265503
--epoch:14030,loss:4.819417476654053,error:2.145655632019043
--epoch:14040,loss:5.0184431076049805,error:1.9869921207427979
--epoch:14050,loss:2.375135898590088,error:1.5250728130340576
--epoch:14060,loss:3.68145489692688,error:1.8276374340057373
--epoch:14070,loss:4.512950897216797,error:2.0271031856536865
--epoch:14080,loss:3.5556023120880127,error:1.8320629596710205
--epoch:14090,loss:4.615334987640381,error:2.085817337036133
--epoch:14

--epoch:15260,loss:5.465040683746338,error:2.1457672119140625
--epoch:15270,loss:3.303042411804199,error:1.7524305582046509
--epoch:15280,loss:2.899378776550293,error:1.6216686964035034
--epoch:15290,loss:3.2008731365203857,error:1.7614909410476685
--epoch:15300,loss:2.8599629402160645,error:1.5980195999145508
--epoch:15310,loss:2.637752056121826,error:1.494600772857666
--epoch:15320,loss:4.595826625823975,error:1.8499234914779663
--epoch:15330,loss:2.5238025188446045,error:1.5506126880645752
--epoch:15340,loss:3.8000974655151367,error:1.7270689010620117
--epoch:15350,loss:3.5196328163146973,error:1.6885275840759277
--epoch:15360,loss:2.8710548877716064,error:1.6723475456237793
--epoch:15370,loss:3.419110059738159,error:1.7568696737289429
--epoch:15380,loss:2.714279890060425,error:1.6249313354492188
--epoch:15390,loss:3.206355571746826,error:1.7812243700027466
--epoch:15400,loss:4.037102222442627,error:1.8181220293045044
--epoch:15410,loss:3.299248695373535,error:1.7232774496078491
--e

--epoch:16570,loss:2.994333267211914,error:1.616428256034851
--epoch:16580,loss:3.665144205093384,error:1.7453161478042603
--epoch:16590,loss:2.1743929386138916,error:1.4034234285354614
--epoch:16600,loss:3.3436214923858643,error:1.7532087564468384
--epoch:16610,loss:3.7492921352386475,error:1.8956093788146973
--epoch:16620,loss:2.8478949069976807,error:1.5687997341156006
--epoch:16630,loss:4.263401985168457,error:1.7944130897521973
--epoch:16640,loss:3.7354624271392822,error:1.9160315990447998
--epoch:16650,loss:3.9421372413635254,error:1.7162097692489624
--epoch:16660,loss:2.926135540008545,error:1.5628957748413086
--epoch:16670,loss:3.373612403869629,error:1.698329210281372
--epoch:16680,loss:3.220856189727783,error:1.8010945320129395
--epoch:16690,loss:3.230409622192383,error:1.7151825428009033
--epoch:16700,loss:2.836491584777832,error:1.6492669582366943
--epoch:16710,loss:3.976757049560547,error:1.8350136280059814
--epoch:16720,loss:4.057581424713135,error:1.8475966453552246
--ep

In [ ]:
#i_ 53999

In [ ]:
#run_time=time.time()-s_time
print('Trainning Time:{:.0f} h,{:.0f} min,{:.2f} sec'.format(run_time//3600,(run_time%3600)//60,run_time%60))

In [ ]:
#保存模型
saver=tf.train.Saver()
saver.save(sess,'./model_save/model_err17.ckpt',global_step=130*60*3)

In [ ]:
#加载模型
saver=tf.train.Saver()
saver.restore(sess,'./model_save/model_mit/model_err106.ckpt-44270')

In [ ]:
batch_size=128
mask=np.random.choice(range(5000),batch_size,replace=False)
face_,face_mask_=data['val_face'][mask],data['val_face_mask'][mask]
left_eye_,right_eye_=data['val_eye_left'][mask],data['val_eye_right'][mask]
y_=data['val_y'][mask]
feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,FaceMask:face_mask_,\
          Y:y_,LR:1e-3}
loss_,m_=sess.run([Loss,merge],feed_dict=feed_dict)

In [ ]:
data={}
mean_val=[]
file_addr=r'D:\Data warehouse\Face_Picture\eye_tracker_train_and_val'
for i in [ 'val_y.npy',
          'val_eye_left.npy',
          'val_eye_right.npy',
          'val_face.npy',
          'val_face_mask.npy',
          'train_y.npy']:
    d_=np.load(os.path.join(file_addr,i))
    ori_shape=d_.shape
    d_=d_.reshape(ori_shape[0],-1).astype('float32')
    if i[-5] !='y':
        #使得所有值小于 1
        d_/=255.0
        #使得均值为0
        mean_=np.mean(d_,axis=0)
        d_-=mean_
        mean_val.append(mean_)
    data[i[:-4]]=d_.reshape(ori_shape)
print('Load Data Done~')

In [ ]:
data['train_y'].shape

In [ ]:
np.min(data['val_y'][:,0]),np.min(data['val_y'][:,1])

In [ ]:
np.max(data['val_y'][:,0]),np.max(data['val_y'][:,1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(8,5), dpi=380)
plt.scatter(data['train_y'][:,0],data['train_y'][:,1],1,label='train',c=np.arctan2(data['train_y'][:,1],data['train_y'][:,0]))
plt.scatter(data['val_y'][:,0],data['val_y'][:,1],5,label='val',c=np.arctan2(data['val_y'][:,1],data['val_y'][:,0]))
plt.legend()
plt.plot()
plt.show()